## tradeSeq for DE along fallopian tube axis (epithelial spots) in female reproductive tract

In [ ]:
#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("tradeSeq")

In [ ]:
library(tradeSeq)
library(SingleCellExperiment)
library(slingshot)

In [ ]:
adata <- readRDS("//lustre/scratch126/cellgen/team292/vl6/VISIUM/femalereproductiveaxis_epi_cytassist_downsampled_fallopiantube.rds")
adata


In [ ]:
counts <- assay(adata, "X")

In [ ]:
set.seed(7)
pseudotime <- colData(adata)$FemaleReproductiveAxis
cellWeights <- rep(1, 88)

In [ ]:
#icMat <- evaluateK(counts = counts, sds = SlingshotDataSet(adata), k = 3:10, 
#                   nGenes = 200, verbose = T)

In [ ]:
library(scran)

In [ ]:
libsizes <- colSums(counts)
size.factors <- libsizes/mean(libsizes)
logcounts(adata) <- log2(t(t(counts)/size.factors) + 1)
assayNames(adata)

In [ ]:
# Filter genes with non-zero biological components
getVariableGenes <- function(adata,fdr_thresh=NULL,var_thresh=0) {
  gene_var <- modelGeneVar(adata)
  genes <- getTopHVGs(gene_var, var.threshold =var_thresh ,fdr.threshold = fdr_thresh)
  return(genes)
}

In [ ]:
genes <- getVariableGenes(adata)

In [ ]:
length(genes)

In [ ]:
dim(counts[genes,])

In [ ]:
sce <- fitGAM(counts = counts[genes,], pseudotime = pseudotime, cellWeights = cellWeights, nknots = 5,
                verbose = TRUE, parallel = TRUE)

In [ ]:
table(rowData(sce)$tradeSeq$converged)

In [ ]:
adata <- adata[genes, ]

In [ ]:
adata

### ASSOCIATION TEST

A first exploration of the data analysis may consist of checking whether gene expression is associated with a particular lineage. The statistical test performed here, implemented in the associationTest function, is testing the null hypothesis that all smoother coefficients are equal to each other. This can be interpreted as testing whether the average gene expression is significantly changing along pseudotime.



In [ ]:
assoRes <- associationTest(sce)
head(assoRes)

In [ ]:
write.csv(assoRes, file = '/nfs/team292/vl6/FetalReproductiveTract/manuscript_v1/supp_tables/SupplementaryTable8_fallopian_epi_cytassist.csv')


In order to discover marker genes of the progenitor or differentiated cell population, researchers may be interested in assessing differential expression between the progenitor cell population (i.e., the starting point of a lineage) with the differentiated cell type population (i.e., the end point of a lineage). The function startVsEndTest uses a Wald test to assess the null hypothesis that the average expression at the starting point of the smoother (progenitor population) is equal to the average expression at the end point of the smoother (differentiated population). The test basically involves a comparison between two smoother coefficients for every lineage. The function startVsEndTest performs a global test across all lineages by default (i.e. it compares the start and end positions for all lineages simultaneously), but you can also assess all lineages separately by setting lineages=TRUE. Below, we adopt an omnibus test across the two lineages.

### Genes strongly associated with pseudotime 

In [ ]:
assoRes = assoRes[assoRes$pvalue < 0.001,]

In [ ]:
dim(assoRes)

In [ ]:
assoRes['PNOC', ]

In [ ]:
assoRes['CRTAC1', ]

In [ ]:
assoRes['LYPD1', ]

In [ ]:
assoRes['ERP27', ]

In [ ]:
assoRes['ROS1', ]

In [ ]:
assoRes['MUC6', ]

In [ ]:
assoRes = assoRes[assoRes$meanLogFC > 0.5, ]

In [ ]:
dim(assoRes)

In [ ]:
assoRes = assoRes[assoRes$waldStat > 20, ]

In [ ]:
dim(assoRes)

In [ ]:
write.csv(assoRes, file = "/lustre/scratch126/cellgen/team292/vl6/VISIUM/FallopianAxis_epi_genes.csv",
          row.names = TRUE)

In [ ]:
assoRes['PNOC', ]

In [ ]:
assoRes['CRTAC1', ]

In [ ]:
assoRes['LYPD1', ]

In [ ]:
assoRes['ERP27', ]

In [ ]:
assoRes['ROS1', ]

In [ ]:
assoRes['MUC6', ]

In [ ]:
assoRes = assoRes[order(assoRes$waldStat, decreasing = TRUE),]

In [ ]:
# Get the index of the DataFrame
spatially_variable = c('PNOC', 'MUC6', 'HAPLN3', 'KCNN4', 'GATA6', 'AFAP1L2', 'NOTUM', 'RSPO1', 'RNASE1', 'CRTAC1', 'WT1',
                       'ASS1', 'DLGAP1', 'TMEM132B', 'RGL3', 'WDR72', 'ADAM28', 'EGR1', 'KCP', 'ADAMTS1', 'CHRNA4', 'CCN2', 
                       'LYPD1', 'CD109', 'DOK5', 'MLXIPL', 'ETV4', 'TTYH1', 'PPP2R2B', 'CCN1', 'MCL1', 'PELI2', 'SLC14A1', 
                       'NPR1', 'SHANK1', 'PCDH17', 'JUN', 'CLDN6', 'FGFR3', 'FGF17', 'FOSB', 'ASCL2', 'ALDH1A2', 'PADI2', 
                       'HS6ST1', 'PRSS22', 'IRGQ', 'CPXM2', 'TJAP1', 'LHX1', 'CDH22', 'IMPG2', 'BTG2', 'IQCN', 'HOXB3', 'TRIB1',
                       'MMP28', 'SLC7A1', 'CSRNP1', 'EGR2', 'CEMIP', 'APOA1', 'AMOTL2', 'ATF3', 'SLITRK2', 'GGT7', 'NR4A1',
                       'COX4I1', 'TRMT61A', 'EYA4', 'TSC22D1', 'NFKBIA', 'PALM3', 'MAL2', 'INPPL1', 'MROH1', 'OGT', 'AC008397.1',
                       'ST6GALNAC5', 'MMP7', 'BHLHE40', 'GPT', 'WNT6', 'SELENOH', 'ZBED6CL', 'KDM3B', 'LGR5', 'SLC34A2', 'AFAP1L1', 
                       'RBM22', 'SLBP', 'BCAT1', 'EPS8L2', 'GALNT17', 'FRZB', 'KRT18', 'MPP7', 'RTN3', 'SV2C', 'URI1', 'SLPI',
                       'TRAPPC3', 'NDUFA1', 'SP5', 'CARD10', 'CDK16', 'SCRIB', 'NOL4L', 'NME1', 'OBSCN', 'FBN3', 'S100A1', 'CD9', 
                       'HIC2', 'PSAT1', 'SLC39A10', 'MFAP5', 'CLIC5', 'ADAMTS6', 'PALMD', 'GTF2IRD1', 'CPLX2', 'FGFRL1', 'MAP3K21',
                       'TM4SF1', 'MARCH7', 'NHP2', 'PM20D2', 'STOML2', 'ABCB7', 'PSME1', 'MXRA5', 'SET', 'UQCR10', 'VSTM2B', 'TBC1D24', 
                       'PRR36', 'SRSF9', 'MAGT1', 'SLC25A25', 'LCN12', 'SMARCC1', 'ZNF503', 'SYF2', 'SLC26A7', 'TMED3', 'FAM3B', 'AHCY',
                       'KLHL12', 'HIGD2A', 'CHTF18', 'EME2', 'HNRNPL', 'NELL1', 'SERINC5', 'CMTM6', 'S100A13', 'TARBP1', 'BMPR1B', 'CDH6',
                       'CREBZF', 'LARS', 'BICRAL', 'IL1R1', 'CHCHD3', 'ERP27', 'SENP6', 'RHOT2', 'APH1A', 'CELSR1', 'NONO', 'SLC27A1',
                       'PPHLN1', 'ERG28', 'SMAD4', 'SMIM15', 'ADAMTS15', 'LMF1', 'FDFT1', 'KLK8', 'KLF10', 'SERBP1', 'ADGRB1', 'IMPACT',
                       'BABAM2', 'YAP1', 'INTS14', 'OCLN', 'CREG1', 'PPP1R13L', 'RGS10', 'ETFA', 'SLC38A1', 'NCOA7', 'SCAF1', 'ATP1A1', 
                       'NCKIPSD', 'ZNF148', 'PRIM2', 'DLGAP3', 'CCNB1IP1', 'PPP1R16A', 'PGD', 'NDUFB9', 'NUCKS1', 'MAK16', 'PNRC2',
                       'GCHFR', 'ARHGEF3', 'BLACAT1', 'EIF4A1', 'AGER', 'EXD3', 'PKHD1', 'TELO2', 'HNRNPH2',
                       'EVPL', 'PRKDC', 'SDHB', 'ANAPC16', 'APEX1', 'TBC1D5', 'DYM', 'GUCY1B1', 'MAFF', 'FAM71A', 'MAP3K15')

In [ ]:
library(ggplot2)

In [ ]:
for (gene in spatially_variable) {
    print(gene)
    print(plotSmoothers(sce, counts, gene = gene, alpha = 0) + ggtitle(gene) +
                          theme(title = element_text(size = 20, face = 'bold'),
                                axis.title = element_text(size = 20),
                                axis.text = element_text(size = 20),
                                legend.text = element_text(size = 20),
                                legend.title = element_text(size = 20, face = "bold")))
}

In [ ]:
model_vals_df <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100) # output is 708 genes x 100 pseudospace values 

In [ ]:
model_vals_mtx <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100, tidy=FALSE)

In [ ]:
dim(model_vals_mtx) # 1207 genes x 100 pseudospace values 

In [ ]:
write.csv(model_vals_mtx, file = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/fallopian_axis_visium_fitted_values_tradeseq_epithelium.csv')


In [ ]:
erp27 <- plotSmoothers(sce, counts, gene = 'ERP27', alpha=0, 
                    xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("ERP27") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 3)
erp27

In [ ]:
pnoc <- plotSmoothers(sce, counts, gene = 'PNOC', alpha =0, 
                     xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("PNOC") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 5)
pnoc

In [ ]:
ttyh1 <- plotSmoothers(sce, counts, gene = 'TTYH1', alpha =0, 
                     xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("TTYH1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 3.5)
ttyh1

In [ ]:
wt1 <- plotSmoothers(sce, counts, gene = 'WT1', alpha=0, 
                     xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("WT1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 5)
wt1

In [ ]:
muc6 <- plotSmoothers(sce, counts, gene = 'MUC6', alpha=0, 
                     xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("MUC6") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 7)
muc6

In [ ]:
alpk1 <- plotSmoothers(sce, counts, gene = 'PAX2', alpha=0) + ggtitle("PAX2") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))
alpk1

In [ ]:
LYPD1 <- plotSmoothers(sce, counts, gene = 'LYPD1', alpha=0) + ggtitle("LYPD1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))
LYPD1

In [ ]:
p <- grid.arrange(pnoc, erp27, WDR72, muc6, nrow = 2, ncol = 2)

In [ ]:
ggsave('/home/jovyan/RepTract/figures-epithelial/visium_fallopiantube_tradeseq.pdf',
       plot = p,
       width = 8, height = 8, dpi = 300)